# GLADE - CMIP5 Database

This notebook demonstrates how to generate CMIP5 database using `intake_cmip` for CMIP5 datasets on NCAR's GLADE file system.

In [1]:
# Import Packages
import os 
from dask.distributed import Client
from dask import delayed
import dask
from pathlib import Path
import re
import pandas as pd
from dask_jobqueue import SLURMCluster
PROJECT = os.environ["PBS_ACCOUNT"]

In [2]:
dask.config.set({'distributed.dashboard.link':'http://localhost:8877/proxy/{port}/status'})

In [3]:
# Create Dask Cluster using Jobqueue
cluster = SLURMCluster(project=PROJECT, processes=6, cores=12, memory="2GB",
                           env_extra=['export LANG="en_US.utf8"',
                                      'export LANGUAGE="en_US.utf8"',
                                      'export LC_ALL="en_US.utf8"',
                                      'export LD_LIBRARY_PATH=""',])

In [4]:
cluster.adapt(minimum=72, maximum=108)

In [5]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1981391       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981392       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981393       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981394       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981395       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981396       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981397       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981398       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981399       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981400       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981401       dav dask-wor  abanihi PD       0:00      1 (Priority)
           1981402       dav dask-wor  abanihi

In [6]:
client = Client(cluster)

In [7]:
client

<Client: scheduler='tcp://10.12.205.200:46459' processes=104 cores=208>

In [8]:
import intake_cmip
from intake_cmip.database import create_cmip5_database

In [9]:
cmip5_root = "/glade/collections/cmip/cmip5" # This is the root dir of CMIP5 datasets on Glade.

In [10]:
%time df = create_cmip5_database(root_dir=cmip5_root)  # Create and persist data in user's home directory

**** Persisting CMIP5 database: /glade/u/home/abanihi/.intake_cmip/cmip5.csv ****
CPU times: user 1min 11s, sys: 4.16 s, total: 1min 15s
Wall time: 8min 2s


In [11]:
df.head()

ensemble  experiment                                      file_basename  \
0   r2i1p1       rcp85      ua_Amon_CanESM2_rcp85_r2i1p1_200601-210012.nc   
1   r5i1p1       rcp85      ua_Amon_CanESM2_rcp85_r5i1p1_200601-210012.nc   
2   r3i1p1  historical  ua_Amon_GFDL-CM3_historical_r3i1p1_200501-2005...   
3   r4i1p1       rcp85      ua_Amon_CanESM2_rcp85_r4i1p1_200601-210012.nc   
4   r1i1p1  historical  ua_Amon_GFDL-CM3_historical_r1i1p1_200501-2005...   

                                       file_fullpath frequency institution  \
0  /glade/collections/cmip/cmip5/output1/CCCma/Ca...       mon       CCCma   
1  /glade/collections/cmip/cmip5/output1/CCCma/Ca...       mon       CCCma   
2  /glade/collections/cmip/cmip5/output1/NOAA-GFD...       mon   NOAA-GFDL   
3  /glade/collections/cmip/cmip5/output1/CCCma/Ca...       mon       CCCma   
4  /glade/collections/cmip/cmip5/output1/NOAA-GFD...       mon   NOAA-GFDL   

      model  realm                                               root varname  \
0   CanESM2  atmos  /glade/collections/cmip/cmip5/output1/CCCma/Ca...      ua   
1   CanESM2  atmos  /glade/collections/cmip/cmip5/output1/CCCma/Ca...      ua   
2  GFDL-CM3  atmos  /glade/collections/cmip/cmip5/output1/NOAA-GFD...      ua   
3   CanESM2  atmos  /glade/collections/cmip/cmip5/output1/CCCma/Ca...      ua   
4  GFDL-CM3  atmos  /glade/collections/cmip/cmip5/output1/NOAA-GFD...      ua   

  version  
0      v0  
1      v0  
2      v0  
3      v0  
4      v0

In [12]:
df.model.nunique()  # Find the total number of unique climate models

55

In [13]:
df.realm.unique()  # find the list of unique realms

array(['atmos', 'seaIce', 'aerosol', 'landIce', 'land', 'ocean',
       'ocnBgchem'], dtype=object)

In [14]:
df.varname.nunique() # How many unique variables

454

In [15]:
df.tail()

ensemble  experiment                                      file_basename  \
87560   r1i1p1       rcp85  tasmax_day_ACCESS1-3_rcp85_r1i1p1_20810101-210...   
87561   r1i1p1       rcp85  tas_day_ACCESS1-3_rcp85_r1i1p1_20810101-210012...   
87562   r1i1p1  historical  ua_day_ACCESS1-0_historical_r1i1p1_20050101-20...   
87563   r1i1p1       rcp85  va_day_ACCESS1-3_rcp85_r1i1p1_20960101-2100123...   
87564   r1i1p1  historical  va_day_ACCESS1-0_historical_r1i1p1_20050101-20...   

                                           file_fullpath frequency  \
87560  /glade/collections/cmip/cmip5/output1/CSIRO-BO...       day   
87561  /glade/collections/cmip/cmip5/output1/CSIRO-BO...       day   
87562  /glade/collections/cmip/cmip5/output1/CSIRO-BO...       day   
87563  /glade/collections/cmip/cmip5/output1/CSIRO-BO...       day   
87564  /glade/collections/cmip/cmip5/output1/CSIRO-BO...       day   

      institution      model  realm  \
87560   CSIRO-BOM  ACCESS1-3  atmos   
87561   CSIRO-BOM  ACCESS1-3  atmos   
87562   CSIRO-BOM  ACCESS1-0  atmos   
87563   CSIRO-BOM  ACCESS1-3  atmos   
87564   CSIRO-BOM  ACCESS1-0  atmos   

                                                    root varname version  
87560  /glade/collections/cmip/cmip5/output1/CSIRO-BO...  tasmax      v4  
87561  /glade/collections/cmip/cmip5/output1/CSIRO-BO...     tas      v4  
87562  /glade/collections/cmip/cmip5/output1/CSIRO-BO...      ua      v4  
87563  /glade/collections/cmip/cmip5/output1/CSIRO-BO...      va      v4  
87564  /glade/collections/cmip/cmip5/output1/CSIRO-BO...      va      v4

In [16]:
%time len(df) # Find the total number of files available in generated CMIP5 database

CPU times: user 23 µs, sys: 2 µs, total: 25 µs
Wall time: 35.8 µs


87565

In [17]:
df.groupby('model').nunique()

ensemble  experiment  file_basename  file_fullpath  frequency  \
model                                                                           
ACCESS1-0              3           3            190            190          4   
ACCESS1-3              3           6            213            213          4   
ACCESS1.0              2           3              3              3          1   
BNU-ESM                1           6             75             75          4   
CCSM4                 87          44          55336          55336          5   
CESM1-BGC             15          12           3804           3804          4   
CESM1-CAM5            37          16           7075           7075          5   
CESM1-FASTCHEM         5           2            633            633          3   
CESM1-WACCM           12           5           3214           3214          3   
CMCC-CESM              1           2             75             75          2   
CMCC-CM                1           4             50             50          4   
CMCC-CMS               1           4             45             45          2   
CNRM-CM5              10           4            291            291          4   
CNRM-CM5-2             1           1             15             15          1   
CSIRO-Mk3-6-0         42           8           1233           1233          3   
CSIRO-Mk3L-1-2         3           2             48             48          1   
CanCM4                10           2            150            150          1   
CanESM2               17          11            839            839          3   
EC-EARTH              13           3            343            343          3   
FGOALS-g2              5           6            135            135          4   
FGOALS-s2              3           2             36             36          1   
FIO-ESM                3           6             73             73          1   
GFDL-CM2p1            10           1             40             40          1   
GFDL-CM3               5           4            457            457          4   
GFDL-ESM2G             3           8            547            547          5   
GFDL-ESM2M             8           9            624            624          5   
GISS-E2-H            113           8           1762           1762          4   
GISS-E2-H-CC           1           3             73             73          1   
GISS-E2-R            137           8           1939           1939          4   
GISS-E2-R-CC           1           3             73             73          1   
HadCM3                10           2            404            404          2   
HadGEM2-A              1           1              5              5          1   
HadGEM2-AO             1           3             48             48          2   
HadGEM2-CC             3           3            456            456          3   
HadGEM2-ES             5           6           1144           1144          5   
IPSL-CM5A-LR          20           8           1410           1410          5   
IPSL-CM5A-MR           6           4            339            339          4   
IPSL-CM5B-LR           1           3            140            140          3   
MIROC-ESM              3           8            349            349          5   
MIROC-ESM-CHEM         9           5            200            200          4   
MIROC4h                3           2            205            205          2   
MIROC5                 5           5            839            839          4   
MPI-ESM-LR             3           9            905            905          4   
MPI-ESM-MR             3           4            253            253          3   
MPI-ESM-P              2           1             32             32          2   
MRI-AGCM3-2H           1           1              5              5          1   
MRI-AGCM3-2S           1           1              3              3          1   
MRI-CGCM3              5           4           

In [18]:
%load_ext watermark

In [27]:
%watermark --iversion -g -h -m -v -u -d

dask        0.20.0
re          2.2.1
pandas      0.23.4
intake_cmip  0+untagged.77.gf8d74ed.dirty
last updated: 2018-12-26 

CPython 3.6.6
IPython 7.0.1

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 3.10.0-693.21.1.el7.x86_64
machine    : x86_64
processor  : x86_64
CPU cores  : 72
interpreter: 64bit
host name  : casper01
Git hash   : f8d74ed4e3ee566582ed98a542584338b04e1d4b
